In [40]:
import pandas as pd
import duckdb

import dash
from dash import Dash, dcc, html, Input, Output

import plotly.express as px

In [37]:
# Caminho do banco
db_path = "../data/duckdb/database.duckdb"

# Conexão com o banco DuckDB
con = duckdb.connect(db_path)

# Carrega os dados de feature engineering
df_previsao = con.execute("""
    SELECT 
        *
                 
    FROM output.previsao_consumo
""").df()

df_consumo= con.execute("""
    SELECT 
          date
        , client_id
        , region
        , consumption_kwh        
    FROM gold.consumo_geral
""").df()

In [ ]:
dia_previsto = df_consumo['date'].max() + pd.Timedelta(days=1)

df_previsao['date'] = dia_previsto

df_previsao = df_previsao.rename({'forecast' : 'consumption_kwh'},axis=1)

df = pd.concat([df_consumo,df_previsao]).sort_values('date')

df['date'] = pd.to_datetime(df['date'])

df['type'] = df['date'].apply(lambda x: 'Forecast' if x == dia_previsto else 'Real')

In [54]:
dias_semana_map = {
    0: 'Segunda',
    1: 'Terça',
    2: 'Quarta',
    3: 'Quinta',
    4: 'Sexta',
    5: 'Sábado',
    6: 'Domingo'
}

In [55]:
# Inicializa app
app = Dash(__name__)
app.title = "Dashboard de Consumo"

# Lista de regiões únicas ordenadas
region_options = sorted(df['region'].dropna().unique())

app.layout = html.Div([
    html.H2("Consumo de Energia por Cliente"),

    html.Div([
        html.Label("Região"),
        dcc.Dropdown(
            id='region-filter',
            options=[{'label': r, 'value': r} for r in region_options],
            value=region_options[0],
            clearable=False
        ),

        html.Label("Cliente"),
        dcc.Dropdown(
            id='client-filter',
            multi=True,
            placeholder="Selecione um ou mais clientes"
        )
    ], style={'width': '40%', 'display': 'inline-block'}),

    dcc.Graph(id='consumo-graph'),
    html.Hr(),
    html.H2("Ciclo de Consumo por Cliente"),
    dcc.Graph(id='ciclo-consumo-graph'),
    html.Hr(),
    html.H2("Total Previsto por Região"),
    dcc.Graph(id='total-regiao-graph'),
    html.H2("Ciclo de Consumo por Região"),
    dcc.Graph(id='ciclo-regiao-graph')
])

@app.callback(
    Output('client-filter', 'options'),
    Output('client-filter', 'value'),
    Input('region-filter', 'value')
)
def update_client_dropdown(selected_region):
    clientes_region = df[df['region'] == selected_region]['client_id'].unique()
    options = [{'label': c, 'value': c} for c in sorted(clientes_region)]
    return options, [clientes_region[0]] if len(clientes_region) > 0 else []

@app.callback(
    Output('consumo-graph', 'figure'),
    Input('region-filter', 'value'),
    Input('client-filter', 'value')
)
def update_graph(selected_region, selected_clients):
    filtered_df = df[df['region'] == selected_region]

    if selected_clients:
        filtered_df = filtered_df[filtered_df['client_id'].isin(selected_clients)]

    forecast_df = filtered_df[filtered_df['type'] == 'Forecast']
    real_df = filtered_df[filtered_df['type'] == 'Real']

    fig = px.line(
        real_df,
        x='date',
        y='consumption_kwh',
        color='client_id',
        title='Consumo Real e Previsto por Cliente',
        labels={'consumption_kwh': 'Consumo (kWh)'}
    )

    fig.add_scatter(
        x=forecast_df['date'],
        y=forecast_df['consumption_kwh'],
        mode='markers',
        name=f"Previsto",
        marker=dict(color='red', symbol='circle')
    )

    fig.update_layout(template='plotly_white')
    return fig

@app.callback(
    Output('total-regiao-graph', 'figure'),
    Input('region-filter', 'value')
)
def update_total_regiao(selected_region):
    total_df = df[df['type'] == 'Forecast']
    grouped = total_df.groupby('region')['consumption_kwh'].sum().reset_index()

    fig = px.bar(
        grouped,
        x='region',
        y='consumption_kwh',
        title='Total Previsto de Consumo por Região',
        labels={'consumption_kwh': 'Consumo Previsto (kWh)', 'region': 'Região'},
        text_auto='.2s',
        color='region'
    )
    fig.update_layout(template='plotly_white')
    return fig

@app.callback(
    Output('ciclo-consumo-graph', 'figure'),
    Input('region-filter', 'value'),
    Input('client-filter', 'value')
)
def update_ciclo_graph(selected_region, selected_clients):
    filtered_df = df[(df['region'] == selected_region) & (df['type'] == 'Real')]

    if selected_clients:
        filtered_df = filtered_df[filtered_df['client_id'].isin(selected_clients)]

    filtered_df['dia_semana'] = pd.to_datetime(filtered_df['date']).dt.dayofweek

    filtered_df['nome_dia'] = filtered_df['dia_semana'].map(dias_semana_map)

    ciclo_df = filtered_df.groupby(['client_id', 'nome_dia'])['consumption_kwh'].mean().reset_index()

    # Para garantir a ordem correta dos dias
    ordem_dias = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo']
    ciclo_df['nome_dia'] = pd.Categorical(ciclo_df['nome_dia'], categories=ordem_dias, ordered=True)

    fig = px.line(
        ciclo_df,
        x='nome_dia',
        y='consumption_kwh',
        color='client_id',
        labels={'nome_dia': 'Dia da Semana', 'consumption_kwh': 'Média Consumo (kWh)'},
        title='Ciclo Semanal de Consumo por Cliente'
    )

    fig.update_layout(template='plotly_white')
    return fig

@app.callback(
    Output('ciclo-regiao-graph', 'figure'),
    Input('region-filter', 'value')
)
def update_ciclo_regiao(selected_region):
    filtered_df = df[(df['type'] == 'Real')]

    filtered_df['dia_semana'] = pd.to_datetime(filtered_df['date']).dt.dayofweek

    filtered_df['nome_dia'] = filtered_df['dia_semana'].map(dias_semana_map)

    ciclo_regiao_df = filtered_df.groupby(['region', 'nome_dia'])['consumption_kwh'].mean().reset_index()

    ordem_dias = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo']
    ciclo_regiao_df['nome_dia'] = pd.Categorical(ciclo_regiao_df['nome_dia'], categories=ordem_dias, ordered=True)

    fig = px.line(
        ciclo_regiao_df,
        x='nome_dia',
        y='consumption_kwh',
        color='region',
        labels={'nome_dia': 'Dia da Semana', 'consumption_kwh': 'Média Consumo (kWh)'},
        title='Ciclo Semanal de Consumo por Região'
    )

    fig.update_layout(template='plotly_white')
    return fig


if __name__ == '__main__':
    app.run(debug=True)


C:\Users\JoaoGuelfi\AppData\Local\Temp\ipykernel_14068\3753722907.py:143: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\JoaoGuelfi\AppData\Local\Temp\ipykernel_14068\3753722907.py:145: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [1]:
# O consumo previsto total por região permite:
# Calcular receita do consumo e preço.
# Criar um orçamento e controle financeiro pois ajuda na elaboração de cenários e controle de metas financeiras.
# Alocamento de recursos técnicos, equipes de manutenção, expansão de rede ou instalação de equipamentos.
# Antecipar gargalos operacionais. Se a previsão aponta aumento rápido em alguma região.
# Identifica padrões de consumo como os dias que o cliente mais consome.
# Apoio à precificação dinâmica para ajustar tarifas ou incentivos com base na previsão de demanda.